# Solving a Sudoku with AI
## Coding the board

In [1]:
rows = 'ABCDEFGHI'
cols = '123456789'

def cross(a, b):
    """
    Return a list with all concatenations of a letter in
    `a` with a letter in `b`
    
    Args:
        a: A string
        b: A string
    Returns:
        A list formed by all the possible concatenations of a
        letter in `a` with a letter in `b`
    """
    return [s + t for s in a for t in b]

boxes = cross(rows, cols)
print(boxes)

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9']


In [9]:
# Lets get all the row units
# row_units[0] = ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']
row_units = [cross(row, cols) for row in rows]

# Same for column units
# column_units[0] = ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']
column_units = [cross(rows, col) for col in cols]

# And now for square units
# square_units[0] = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']
square_units = [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')]

unitlist = row_units + column_units + square_units

print('First row unit: ', row_units[0])
print('First col unit: ', column_units[0])
print('First sqr unit: ', square_units[0])

First row unit:  ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']
First col unit:  ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']
First sqr unit:  ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']


In [24]:
def display(values):
    """
    Prints the values of a sudoku as a 2-D grid.
    
    Args:
        values: A dict representing a sodoku.
        
    Returns:
        `None`
    """
    width = 1 + max(len(values[s]) for s in boxes)
    line = '+'.join(['-' * (width * 3)] * 3)
    for row in rows:
        print(''.join(values[row + col].center(width) + ('|' if col in '36' else '') for col in cols))
        if row in 'CF': print(line)
    return

def grid_values(grid):
    """
    Returns a dict that represents a sudoku.
    
    Args:
        grid: A string with the starting numbers
        for all the boxes in a sudoku. Empty boxes
        can be represented as dots `.`.
        Example: `'..3.2.6.'...`
        
    Returns:
        A dict that represents a sudoku. The keys
        will be the boxes labels and it's value will be the number
        or a dot `.` if the box is empty.
    """
    assert len(grid) == 81, "The lenght of `grid` should be 81. A 9x9 sudoku"
    chars = []
    for char in grid:
        chars.append(char)
    return dict(zip(boxes, chars))

display(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))

. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 
